# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos numa importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

### Passo a Passo
- Pegar a cotação do Dólar
- Pegar a cotação do Euro
- Pegar a cotação do Ouro
- Importar e Atualizar a Base de Dados
- Recalcular os preços
- Exportar a base atualizada

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True
# navegador = webdriver.Chrome(options=chrome_options)


# Criando o navegador
navegador = webdriver.Chrome()

# Pegando a cotação do Dólar
navegador.get('https://www.google.com/')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar',Keys.ENTER)
cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Pegando a cotação do Euro
navegador.get('https://www.google.com/')
navegador.find_element('xpath','/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro',Keys.ENTER)
cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# Pegando a cotação do Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')

navegador.quit()

### Agora vamos atualiza a  base de preços com as novas cotações


In [6]:
import pandas as pd


# Importando a Base de Dados
df = pd.read_excel('Produtos.xlsx')

# Atualizando cotação
df.loc[df['Moeda'] == 'Dólar','Cotação'] = float(cotacao_dolar)
df.loc[df['Moeda'] == 'Euro','Cotação'] = float(cotacao_euro)
df.loc[df['Moeda'] == 'Ouro','Cotação'] = float(cotacao_ouro.replace(',','.'))

# Atualizando preço de compra e venda
df['Preço de Compra'] = df['Preço Original'] * df['Cotação']
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']

### Agora vamos exportar a nova base de preços atualizada

In [7]:
df.to_excel('Produtos_atualizado.xlsx', index = False)
nova_tabela = pd.read_excel('Produtos_atualizado.xlsx')
display(nova_tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.096000,5095.94904,1.40,7134.328656
1,Carro Renault,4500.00,Euro,5.528396,24877.78020,2.00,49755.560400
2,Notebook Dell,899.99,Dólar,5.096000,4586.34904,1.70,7796.793368
3,IPhone,799.00,Dólar,5.096000,4071.70400,1.70,6921.896800
4,Carro Fiat,3000.00,Euro,5.528396,16585.18680,1.90,31511.854920
5,Celular Xiaomi,480.48,Dólar,5.096000,2448.52608,2.00,4897.052160
6,Joia 20g,20.00,Ouro,315.120000,6302.40000,1.15,7247.760000
